# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key



In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,newport,51.5877,-2.9984,20.43,74,90,4.73,GB,1691407161
1,1,yellowknife,62.4560,-114.3525,12.29,76,75,3.09,CA,1691407161
2,2,laojunmiao,39.8333,97.7333,29.19,18,100,8.37,CN,1691407161
3,3,dudinka,69.4058,86.1778,25.90,34,3,4.50,RU,1691406893
4,4,paphos,34.7667,32.4167,31.24,66,20,7.72,CY,1691407162


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot_City_data = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 700,
    frame_height = 500,  
    color = "City",
    size = "Humidity"
)

# Display the map
map_plot_City_data

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city = city_data_df[(city_data_df['Wind Speed'] < 4) & (city_data_df['Max Temp'] > 25) & (city_data_df['Humidity'] >= 50)]

# Drop any rows with null values
ideal_city.dropna(axis=0, how='any')

# Display sample data
ideal_city.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,miri,4.4148,114.0089,29.08,84,20,1.54,MY,1691407166
28,28,lazaro cardenas,17.9583,-102.2000,25.82,78,78,2.16,MX,1691407168
36,36,pokhara,28.2333,83.9833,26.45,87,100,1.01,NP,1691407090
37,37,san pedro pochutla,15.7333,-96.4667,25.40,88,10,1.78,MX,1691407170
45,45,manokwari,-0.8667,134.0833,26.95,78,80,1.00,ID,1691407173


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city[['City','Lat', 'Lng', 'Humidity', 'Country']]
#hotel_df.head()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

C:\Users\andy.newman\AppData\Local\Temp\ipykernel_14948\3654870670.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Lat,Lng,Humidity,Country,Hotel Name
20,miri,4.4148,114.0089,84,MY,
28,lazaro cardenas,17.9583,-102.2000,78,MX,
36,pokhara,28.2333,83.9833,87,NP,
37,san pedro pochutla,15.7333,-96.4667,88,MX,
45,manokwari,-0.8667,134.0833,78,ID,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "categories":"accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address =  name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
miri - nearest hotel: Kenyalang Suites Miri
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
pokhara - nearest hotel: Hotel Annapurna View Sarangkot
san pedro pochutla - nearest hotel: Olimpia
manokwari - nearest hotel: Hotel Arfak
howrah - nearest hotel: Sun India Guest House
saipan - nearest hotel: Chalan Kanoa Beach Hotel
houma - nearest hotel: Courtyard by Marriott Houma
cururupu - nearest hotel: No hotel found
ishigaki - nearest hotel: アートホテル石垣島
abepura - nearest hotel: Yoyo
kingston - nearest hotel: Aphrodite Hotel
kirensk - nearest hotel: Ивушка
juan griego - nearest hotel: Hotel Descanso Tropical
ore - nearest hotel: No hotel found
ciudad lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
kourou - nearest hotel: SCI Horizon
guerrero negro - nearest hotel: Plaza sal paraiso
alief - nearest hotel: Sleep Inn & Suites Near Westchase
hithadhoo - nearest hotel: Pebbles Inn
batangafo - nearest hotel: No hotel found
manay - nearest hotel: No hotel foun

,City,Lat,Lng,Humidity,Country,Hotel Name
20,miri,4.4148,114.0089,84,MY,Kenyalang Suites Miri
28,lazaro cardenas,17.9583,-102.2000,78,MX,Hotel Sol del Pacífico
36,pokhara,28.2333,83.9833,87,NP,Hotel Annapurna View Sarangkot
37,san pedro pochutla,15.7333,-96.4667,88,MX,Olimpia
45,manokwari,-0.8667,134.0833,78,ID,Hotel Arfak
...,...,...,...,...,...,...
550,katsuura,35.1333,140.3000,92,JP,潮騒の宿
555,progreso,21.2833,-89.6667,92,MX,Centro Vacacional Obrero CTM
558,ta`u,-14.2336,-169.5144,75,AS,No hotel found
566,esmeralda,21.8525,-78.1169,87,CU,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    hover_cols = ['Hotel Name']
)
# Display the map
plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)